# Load the data from RosBag


In [64]:
import pandas as pd
import numpy as np
from scipy.spatial.transform import Rotation as Rot
# Replace with the actual path to your CSV
csv_file = "model_states.csv"

# Read the CSV into a pandas DataFrame
model_state = pd.read_csv(csv_file)

print(model_state.columns)




Index(['%time', 'field.name0', 'field.name1', 'field.name2', 'field.name3',
       'field.pose0.position.x', 'field.pose0.position.y',
       'field.pose0.position.z', 'field.pose0.orientation.x',
       'field.pose0.orientation.y', 'field.pose0.orientation.z',
       'field.pose0.orientation.w', 'field.pose1.position.x',
       'field.pose1.position.y', 'field.pose1.position.z',
       'field.pose1.orientation.x', 'field.pose1.orientation.y',
       'field.pose1.orientation.z', 'field.pose1.orientation.w',
       'field.pose2.position.x', 'field.pose2.position.y',
       'field.pose2.position.z', 'field.pose2.orientation.x',
       'field.pose2.orientation.y', 'field.pose2.orientation.z',
       'field.pose2.orientation.w', 'field.pose3.position.x',
       'field.pose3.position.y', 'field.pose3.position.z',
       'field.pose3.orientation.x', 'field.pose3.orientation.y',
       'field.pose3.orientation.z', 'field.pose3.orientation.w',
       'field.twist0.linear.x', 'field.twist0.line

In [65]:
ground_truth = pd.read_csv("ground_truth.csv")

# See the first few rows
print(ground_truth.columns)

ground_truth

Index(['%time', 'field.header.seq', 'field.header.stamp',
       'field.header.frame_id', 'field.child_frame_id',
       'field.pose.pose.position.x', 'field.pose.pose.position.y',
       'field.pose.pose.position.z', 'field.pose.pose.orientation.x',
       'field.pose.pose.orientation.y', 'field.pose.pose.orientation.z',
       'field.pose.pose.orientation.w', 'field.pose.covariance0',
       'field.pose.covariance1', 'field.pose.covariance2',
       'field.pose.covariance3', 'field.pose.covariance4',
       'field.pose.covariance5', 'field.pose.covariance6',
       'field.pose.covariance7', 'field.pose.covariance8',
       'field.pose.covariance9', 'field.pose.covariance10',
       'field.pose.covariance11', 'field.pose.covariance12',
       'field.pose.covariance13', 'field.pose.covariance14',
       'field.pose.covariance15', 'field.pose.covariance16',
       'field.pose.covariance17', 'field.pose.covariance18',
       'field.pose.covariance19', 'field.pose.covariance20',
       'f

,%time,field.header.seq,field.header.stamp,field.header.frame_id,field.child_frame_id,field.pose.pose.position.x,field.pose.pose.position.y,field.pose.pose.position.z,field.pose.pose.orientation.x,field.pose.pose.orientation.y,...,field.twist.covariance26,field.twist.covariance27,field.twist.covariance28,field.twist.covariance29,field.twist.covariance30,field.twist.covariance31,field.twist.covariance32,field.twist.covariance33,field.twist.covariance34,field.twist.covariance35
0,14012000000,1400,14011000000,world,base_link,-2.652655,-4.780032,11.874030,0.093371,-0.004297,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,14022000000,1401,14021000000,world,base_link,-2.649544,-4.783467,11.874463,0.091112,-0.004513,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,14031000000,1402,14031000000,world,base_link,-2.646444,-4.787083,11.874909,0.088849,-0.004749,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,14041000000,1403,14041000000,world,base_link,-2.643355,-4.790877,11.875366,0.086569,-0.004963,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,14051000000,1404,14051000000,world,base_link,-2.640278,-4.794842,11.875836,0.084286,-0.005194,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7327,87290000000,8727,87281000000,world,base_link,-0.130365,-22.176993,12.000504,-0.004775,-0.002347,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7328,87328000000,8728,87291000000,world,base_link,-0.123218,-22.171268,12.000511,-0.004678,-0.002531,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7329,87328000000,8729,87301000000,world,base_link,-0.116081,-22.165538,12.000518,-0.004521,-0.002779,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7330,87328000000,8730,87311000000,world,base_link,-0.108954,-22.159803,12.000524,-0.004441,-0.002945,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Clean the Data

In [66]:
# Create the subset DataFrame from your model_state DataFrame
subset_columns_state_space = [
    '%time',                  # Timestamp column
    'field.name1',            # Name for the model at index 1 (vessel)
    'field.name3',            # Name for the model at index 3 (drone)
    # Pose data for model 1 (vessel):
    'field.pose1.position.x',
    'field.pose1.position.y',
    'field.pose1.position.z',
    'field.pose1.orientation.x',
    'field.pose1.orientation.y',
    'field.pose1.orientation.z',
    # Pose data for model 3 (drone):
    'field.pose3.position.x',
    'field.pose3.position.y',
    'field.pose3.position.z',
    'field.pose3.orientation.x',
    'field.pose3.orientation.y',
    'field.pose3.orientation.z'
]

vessel_drone = model_state[subset_columns_state_space]

# Rename the columns for clarity:
vessel_drone.rename(columns={
    '%time': 'time',
    'field.name1': 'vessel_name',
    'field.name3': 'drone_name',
    'field.pose1.position.x': 'vessel_pos_x',
    'field.pose1.position.y': 'vessel_pos_y',
    'field.pose1.position.z': 'vessel_pos_z',
    'field.pose1.orientation.x': 'vessel_ori_x',
    'field.pose1.orientation.y': 'vessel_ori_y',
    'field.pose1.orientation.z': 'vessel_ori_z',
    'field.pose3.position.x': 'drone_pos_x',
    'field.pose3.position.y': 'drone_pos_y',
    'field.pose3.position.z': 'drone_pos_z',
    'field.pose3.orientation.x': 'drone_ori_x',
    'field.pose3.orientation.y': 'drone_ori_y',
    'field.pose3.orientation.z': 'drone_ori_z'
}, inplace=True)

# Display the renamed DataFrame
vessel_drone



C:\Users\Zephyrus GA401\AppData\Local\Temp\ipykernel_15796\412505076.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vessel_drone.rename(columns={


,time,vessel_name,drone_name,vessel_pos_x,vessel_pos_y,vessel_pos_z,vessel_ori_x,vessel_ori_y,vessel_ori_z,drone_pos_x,drone_pos_y,drone_pos_z,drone_ori_x,drone_ori_y,drone_ori_z
0,14009000000,vessel,quadrotor,-2.550664,-5.272131,2.159016,-0.016152,-0.018357,-0.033576,-2.654839,-4.777739,11.873734,0.094939,-0.004127,-0.003356
1,14009000000,vessel,quadrotor,-2.550484,-5.272079,2.160047,-0.016137,-0.018316,-0.033574,-2.654526,-4.778061,11.873776,0.094715,-0.004149,-0.003351
2,14009000000,vessel,quadrotor,-2.550304,-5.272027,2.161079,-0.016122,-0.018274,-0.033572,-2.654214,-4.778385,11.873818,0.094491,-0.004172,-0.003345
3,14009000000,vessel,quadrotor,-2.550124,-5.271975,2.162112,-0.016107,-0.018233,-0.033570,-2.653902,-4.778710,11.873860,0.094267,-0.004196,-0.003340
4,14009000000,vessel,quadrotor,-2.549945,-5.271923,2.163146,-0.016092,-0.018191,-0.033568,-2.653590,-4.779038,11.873902,0.094043,-0.004220,-0.003334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73344,87348000000,vessel,quadrotor,0.229913,-21.875387,2.753341,-0.010901,0.000716,0.082355,-0.082686,-22.138547,12.000548,-0.003957,-0.003757,0.000124
73345,87349000000,vessel,quadrotor,0.230477,-21.874834,2.753278,-0.010893,0.000697,0.082362,-0.081978,-22.137972,12.000548,-0.003951,-0.003771,0.000124
73346,87350000000,vessel,quadrotor,0.231042,-21.874281,2.753214,-0.010885,0.000679,0.082369,-0.081270,-22.137396,12.000549,-0.003947,-0.003783,0.000124
73347,87351000000,vessel,quadrotor,0.231606,-21.873729,2.753150,-0.010876,0.000661,0.082375,-0.080563,-22.136821,12.000550,-0.003941,-0.003797,0.000123


In [67]:
subset_columns_drone_marker = [
    # Position of drone to the Marker:
    'field.pose.pose.position.x', 
    'field.pose.pose.position.y',
    'field.pose.pose.position.z',
    # Orientation of drone to the Marker:
    'field.pose.pose.orientation.x',
    'field.pose.pose.orientation.y', 
    'field.pose.pose.orientation.z',
    'field.pose.pose.orientation.w',
]

drone_marker = ground_truth[subset_columns_drone_marker]

# Rename the columns to something more meaningful:
drone_marker.rename(columns={
    'field.pose.pose.position.x': 'drone_marker_pos_x',
    'field.pose.pose.position.y': 'drone_marker_pos_y',
    'field.pose.pose.position.z': 'drone_marker_pos_z',
    'field.pose.pose.orientation.x': 'drone_marker_ori_x',
    'field.pose.pose.orientation.y': 'drone_marker_ori_y',
    'field.pose.pose.orientation.z': 'drone_marker_ori_z',
    'field.pose.pose.orientation.w': 'drone_marker_ori_w'
}, inplace=True)

drone_marker


C:\Users\Zephyrus GA401\AppData\Local\Temp\ipykernel_15796\3158438831.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drone_marker.rename(columns={


,drone_marker_pos_x,drone_marker_pos_y,drone_marker_pos_z,drone_marker_ori_x,drone_marker_ori_y,drone_marker_ori_z,drone_marker_ori_w
0,-2.652655,-4.780032,11.874030,0.093371,-0.004297,-0.003318,0.995617
1,-2.649544,-4.783467,11.874463,0.091112,-0.004513,-0.003262,0.995825
2,-2.646444,-4.787083,11.874909,0.088849,-0.004749,-0.003208,0.996029
3,-2.643355,-4.790877,11.875366,0.086569,-0.004963,-0.003153,0.996228
4,-2.640278,-4.794842,11.875836,0.084286,-0.005194,-0.003101,0.996423
...,...,...,...,...,...,...,...
7327,-0.130365,-22.176993,12.000504,-0.004775,-0.002347,0.000136,0.999986
7328,-0.123218,-22.171268,12.000511,-0.004678,-0.002531,0.000134,0.999986
7329,-0.116081,-22.165538,12.000518,-0.004521,-0.002779,0.000133,0.999986
7330,-0.108954,-22.159803,12.000524,-0.004441,-0.002945,0.000131,0.999986


In [68]:

#Replicate each row exactly 10 times:
rep_factor = round(len(vessel_drone)/len(drone_marker))
drone_marker_scaled = drone_marker.loc[
    np.repeat(drone_marker.index, rep_factor)
].reset_index(drop=True)

drone_marker_scaled

,drone_marker_pos_x,drone_marker_pos_y,drone_marker_pos_z,drone_marker_ori_x,drone_marker_ori_y,drone_marker_ori_z,drone_marker_ori_w
0,-2.652655,-4.780032,11.874030,0.093371,-0.004297,-0.003318,0.995617
1,-2.652655,-4.780032,11.874030,0.093371,-0.004297,-0.003318,0.995617
2,-2.652655,-4.780032,11.874030,0.093371,-0.004297,-0.003318,0.995617
3,-2.652655,-4.780032,11.874030,0.093371,-0.004297,-0.003318,0.995617
4,-2.652655,-4.780032,11.874030,0.093371,-0.004297,-0.003318,0.995617
...,...,...,...,...,...,...,...
73315,-0.101839,-22.154063,12.000531,-0.004308,-0.003167,0.000129,0.999986
73316,-0.101839,-22.154063,12.000531,-0.004308,-0.003167,0.000129,0.999986
73317,-0.101839,-22.154063,12.000531,-0.004308,-0.003167,0.000129,0.999986
73318,-0.101839,-22.154063,12.000531,-0.004308,-0.003167,0.000129,0.999986


In [69]:

# Assume vessel_drone (state_space) is high-frequency DataFrame
# and drone_marker is your low-frequency DataFrame
# For example:
# vessel_drone.shape -> (73349, ...) 
# drone_marker.shape -> (7332, ...)

desired_length = len(drone_marker_scaled)  # e.g., 7332 rows

# Generate uniformly spaced indices between 0 and len(vessel_drone)-1
indices = np.linspace(0, len(vessel_drone) - 1, desired_length, dtype=int)

# Select the rows at those indices to form the downsampled DataFrame
vessel_drone_scaled = vessel_drone.iloc[indices].reset_index(drop=True)


# Assuming vessel_drone and drone_marker have the same number of rows
clean_data_setup = pd.concat([vessel_drone_scaled, drone_marker_scaled], axis=1)

clean_data_setup.columns




Index(['time', 'vessel_name', 'drone_name', 'vessel_pos_x', 'vessel_pos_y',
       'vessel_pos_z', 'vessel_ori_x', 'vessel_ori_y', 'vessel_ori_z',
       'drone_pos_x', 'drone_pos_y', 'drone_pos_z', 'drone_ori_x',
       'drone_ori_y', 'drone_ori_z', 'drone_marker_pos_x',
       'drone_marker_pos_y', 'drone_marker_pos_z', 'drone_marker_ori_x',
       'drone_marker_ori_y', 'drone_marker_ori_z', 'drone_marker_ori_w'],
      dtype='object')

# Validation of physical of Marker fron the many perspectives of views

In [70]:
def compute_marker_from_ship(row, offset=np.array([0, 2.5, 5.02])):
    # Vessel's world coordinate from simulation
    vessel_pos = np.array([row['vessel_pos_x'], row['vessel_pos_y'], row['vessel_pos_z']])
    # Marker world coordinate (ship's view) is vessel position plus the fixed offset.
    return vessel_pos + offset


In [71]:


def compute_marker_from_drone(row):
    # 1) Drone's world position (e.g. from /gazebo/model_states, presumably ENU)
    drone_pos = np.array([
        row['drone_pos_x'],
        row['drone_pos_y'],
        row['drone_pos_z']
    ])
    
    # 2) Drone-to-marker relative position (from ArUco pose estimation)
    pdm = np.array([
        row['drone_marker_pos_x'],
        row['drone_marker_pos_y'],
        row['drone_marker_pos_z']
    ])
    
    # 3) Quaternion from marker->camera (order: x, y, z, w).
    quat = [
        row['drone_marker_ori_x'],
        row['drone_marker_ori_y'],
        row['drone_marker_ori_z'],
        row['drone_marker_ori_w']
    ]
    
    # Convert quaternion to a rotation matrix.
    # If it's marker->camera, invert it (camera->marker) by transposing:
    R_marker_to_cam = Rot.from_quat(quat).as_matrix()
    R_cam_to_marker = R_marker_to_cam.T
    
    # 4) Rotate the relative position to express it in marker frame:
    rotated_pd = R_cam_to_marker.dot(pdm)
    
    # -----------------------------------------------------------------
    # OPTIONAL FIXES: Uncomment if needed
    # -----------------------------------------------------------------
    
    # (A) Z-FLIP for altitude mismatch (e.g. ENU vs. Z-down):
    flip_z = np.diag([1, 1, -1])
    rotated_pd = flip_z.dot(rotated_pd)
    
    # (B) Full NED->ENU remap:
    # R_ned_to_enu = np.array([
    #     [0,  1,  0],  # x_enu = y_ned
    #     [1,  0,  0],  # y_enu = x_ned
    #     [0,  0, -1]   # z_enu = -z_ned
    # ])
    # rotated_pd = R_ned_to_enu.dot(rotated_pd)
    
    # -----------------------------------------------------------------
    
    # 5) Compute the marker's world position:
    marker_pos_drone = drone_pos + rotated_pd
    
    return marker_pos_drone


In [72]:

# - Vessel world position in columns 'vessel_x', 'vessel_y', 'vessel_z'

def compute_marker_positions(row):
    # Compute from ship's view
    marker_from_ship = compute_marker_from_ship(row)  # vector: [x, y, z]
    
    # Compute from drone's view
    marker_from_drone = compute_marker_from_drone(row)  # vector: [x, y, z]
    
    # For example, also compute the error between them:
    error = marker_from_drone - marker_from_ship
    
    return pd.Series({
        'marker_ship_x': marker_from_ship[0],
        'marker_ship_y': marker_from_ship[1],
        'marker_ship_z': marker_from_ship[2],
        'marker_drone_x': marker_from_drone[0],
        'marker_drone_y': marker_from_drone[1],
        'marker_drone_z': marker_from_drone[2],
        
    })
# Apply to your DataFrame
marker_positions = clean_data_setup.apply(compute_marker_positions, axis=1)

# Now you can inspect the result:
print(marker_positions.head())




   marker_ship_x  marker_ship_y  marker_ship_z  marker_drone_x  \
0      -2.550664      -2.772131       7.179016       -5.177675   
1      -2.550484      -2.772079       7.180047       -5.177363   
2      -2.550304      -2.772027       7.181079       -5.177051   
3      -2.550124      -2.771975       7.182112       -5.176739   
4      -2.549945      -2.771923       7.183146       -5.176427   

   marker_drone_y  marker_drone_z  
0       -7.281728       -0.705742  
1       -7.282050       -0.705700  
2       -7.282374       -0.705658  
3       -7.282700       -0.705616  
4       -7.283027       -0.705573  


In [73]:
marker_positions

,marker_ship_x,marker_ship_y,marker_ship_z,marker_drone_x,marker_drone_y,marker_drone_z
0,-2.550664,-2.772131,7.179016,-5.177675,-7.281728,-0.705742
1,-2.550484,-2.772079,7.180047,-5.177363,-7.282050,-0.705700
2,-2.550304,-2.772027,7.181079,-5.177051,-7.282374,-0.705658
3,-2.550124,-2.771975,7.182112,-5.176739,-7.282700,-0.705616
4,-2.549945,-2.771923,7.183146,-5.176427,-7.283027,-0.705573
...,...,...,...,...,...,...
73315,0.229347,-19.375939,7.773403,-0.115543,-44.395733,0.190895
73316,0.229913,-19.375387,7.773341,-0.114835,-44.395158,0.190895
73317,0.230477,-19.374834,7.773278,-0.114127,-44.394582,0.190896
73318,0.231042,-19.374281,7.773214,-0.113420,-44.394007,0.190897


In [105]:

# Define coordinate-wise thresholds (in meters, for example)
# threshold_x = 5
# threshold_y = 5
threshold_z = 4.83


# 'marker_ship_x', 'marker_ship_y', 'marker_ship_z',
# 'marker_drone_x', 'marker_drone_y', 'marker_drone_z'
approximation_data =marker_positions[
    # (np.abs(marker_positions['marker_ship_x'] - marker_positions['marker_drone_x']) < threshold_x) &
    # (np.abs(marker_positions['marker_ship_y'] - marker_positions['marker_drone_y']) < threshold_y) &
    (np.abs(marker_positions['marker_ship_z'] - marker_positions['marker_drone_z']) < threshold_z)
]

print(approximation_data[['marker_ship_x', 'marker_drone_x',
                   'marker_ship_y', 'marker_drone_y',
                   'marker_ship_z', 'marker_drone_z']].head())


       marker_ship_x  marker_drone_x  marker_ship_y  marker_drone_y  \
55050      -1.503563       -2.812786     -17.174365      -38.833898   
55051      -1.503870       -2.813211     -17.175042      -38.834734   
55052      -1.504178       -2.813635     -17.175719      -38.835570   
55060      -1.506631       -2.824042     -17.181124      -38.853360   
55061      -1.506937       -2.824465     -17.181799      -38.854194   

       marker_ship_z  marker_drone_z  
55050       5.178555        0.350017  
55051       5.179146        0.350016  
55052       5.179737        0.350016  
55060       5.184465        0.355146  
55061       5.185056        0.355145  


In [106]:
approximation_data

,marker_ship_x,marker_ship_y,marker_ship_z,marker_drone_x,marker_drone_y,marker_drone_z
55050,-1.503563,-17.174365,5.178555,-2.812786,-38.833898,0.350017
55051,-1.503870,-17.175042,5.179146,-2.813211,-38.834734,0.350016
55052,-1.504178,-17.175719,5.179737,-2.813635,-38.835570,0.350016
55060,-1.506631,-17.181124,5.184465,-2.824042,-38.853360,0.355146
55061,-1.506937,-17.181799,5.185056,-2.824465,-38.854194,0.355145
...,...,...,...,...,...,...
55300,-1.573911,-17.334223,5.330688,-3.085905,-39.316946,0.501431
55301,-1.574165,-17.334824,5.331307,-3.086285,-39.317715,0.501431
55310,-1.576441,-17.340216,5.336890,-3.096331,-39.335935,0.508276
55311,-1.576692,-17.340814,5.337512,-3.096710,-39.336701,0.508275
